In [1]:
import torch
import numpy as np
import random
import os

seed = 2321321

torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

In [2]:
# load entities and relations
import json

path = '../data/'

with open(path+'entities.json', 'r') as fin:
    entities = json.load(fin)
print(len(entities), 'entities')

with open(path+'properties.json', 'r') as fin:
    properties = json.load(fin)
print(len(properties), 'properties')

28497 entities
8913 properties


In [3]:
# load KG
from hdt import HDTDocument, TripleComponentRole

hdt_path = "/ivi/ilps/personal/svakule/"
hdt_file = 'wikidata2018_09_11.hdt'
# hdt_file = 'wikidata20200309.hdt'

PREFIX_E = 'http://www.wikidata.org/entity/'
PREFIX_P = 'http://www.wikidata.org/prop/direct/P'

kg = HDTDocument(hdt_path+hdt_file)

# Display some metadata about the HDT document itself
print("nb triples: %i" % kg.total_triples)
print("nb subjects: %i" % kg.nb_subjects)
print("nb predicates: %i" % kg.nb_predicates)

# load labels
import json

path = '../data/'
max_triples = 50000
offset = 0
namespace = 'predef-wikidata2018-09-all' # 'predef-wikidata2020-03-all'  # 

import sympy

import scipy.sparse as sp


def generate_adj_sp(adjacencies, n_entities, include_inverse):
    '''
    Build adjacency matrix
    '''
    adj_shape = (n_entities, n_entities)
    
    # colect all predicate matrices separately into a list
    sp_adjacencies = []
    for edges in adjacencies:
        # split subject (row) and object (col) node URIs
        n_edges = len(edges)
        row, col = np.transpose(edges)
        
        # duplicate edges in the opposite direction
        if include_inverse:
            _row = np.hstack([row, col])
            col = np.hstack([col, row])
            row = _row
            n_edges *= 2
        
        # create adjacency matrix for this predicate
        data = np.ones(n_edges)
        adj = sp.csr_matrix((data, (row, col)), shape=adj_shape)
        sp_adjacencies.append(adj)
    
    return np.asarray(sp_adjacencies)

nb triples: 2935160017
nb subjects: 760717318
nb predicates: 23387


In [4]:
# from ctc_search import Recognizer
import torch.nn as nn


from ctc_search_new import Recognizer

TOPN = 100  # number of candidate labels to consider as hypotheses space 500

# load pre-trained ASR model
w2v_recognizer = Recognizer(topn=TOPN)

w2v_recognizer.model = nn.DataParallel(w2v_recognizer.model)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# formulate hypotheses: 1k cutoff
def retrieve_answer(audio_file_path, hyp_cutoff=100, rerank_cutoff=0):
    # formulate hypotheses for entities and properties based on BP decoding
    e_hypotheses, p_hypotheses = w2v_recognizer.score_labels_new(audio_file_path)
    e_hypotheses = e_hypotheses.most_common(hyp_cutoff)
    p_hypotheses = p_hypotheses.most_common(hyp_cutoff)
#     print(p_hypotheses)
    # get all entity labels
#     print(e_hypotheses[:2])
    e_hypotheses = [l.upper() for l, c in e_hypotheses]
    # rescoring
    e_rank, loss = w2v_recognizer.rescore_new(e_hypotheses)
#     print(e_rank[:5], loss[:5])
    
    matched_entity_ids = []
    matched_entity_scores = []
    e_ids = []
#     _max = torch.max(loss, dim=-1).values.item()
    _max = max(loss)
#     print(_max)
    for j in range(len(e_rank)):
        label = e_hypotheses[e_rank[j]]
#         score = 1 / (j + 1)
        score = loss[e_rank[j]] / _max #[e_rank[j]].item() ## + 1 #int(-loss[e_rank[j]].item()) / _max# + 1 # hence max score is 2 #1 / (j + 1) #loss[j] #
#         print(score)
        if score > rerank_cutoff:
            # look up entites in the KG by label
            if j < 5:
                print(label, score)
            for e_id in w2v_recognizer.entities_labels2ids[label.lower()]:
                if e_id not in e_ids:
                    e_ids.append(e_id)
                    entity = PREFIX_E + e_id
                    kg_id = kg.string_to_global_id(entity, TripleComponentRole.OBJECT)
                    matched_entity_ids.append(kg_id)
                    matched_entity_scores.append(score)
                    
    p_hypotheses = [l.upper() for l, c in p_hypotheses]
    # rescoring
    p_rank, p_loss = w2v_recognizer.rescore_new(p_hypotheses)
#     _max = torch.max(p_loss, dim=-1).values.item()
    _max = max(p_loss)
#     print(len(p_hypotheses), 'properties')
#     look up predicate ids in the KG
    matched_relation_ids = []
    matched_relation_scores = []
    p_ids = []
    for j in range(len(p_rank)):
        p_label = p_hypotheses[p_rank[j]]
#         score = 1 / (j + 1) #int(-loss[p_rank[j]].item()) / _max + 1 #int(-loss[e_rank[j]].item()) / _max# + 1 # hence max score is 2 #1 / (j + 1) #loss[j] #
        score = p_loss[p_rank[j]] / _max #[p_rank[j]].item()# # + 1 #int(-loss[e_rank[j]].item()) / _max# + 1 # hence max score is 2 #1 / (j + 1) #loss[j] #
        if score > rerank_cutoff:
            # look up entites in the KG by label
            if j < 5:
                print(p_label, score)
            for p_id in w2v_recognizer.relations_labels2ids[p_label.lower()]:
                p_id = p_id[1:]
                if p_id not in p_ids:
                    p_ids.append(p_id)
                    predicate = PREFIX_P + p_id
                    kg_id = kg.string_to_global_id(predicate, TripleComponentRole.PREDICATE)
                    matched_relation_ids.append(kg_id)
                    matched_relation_scores.append(score)

    kg.configure_hops(1, matched_relation_ids, namespace, True, False)
    subgraph = kg.compute_hops(matched_entity_ids, max_triples, offset)
    entity_ids, predicate_ids, adjacencies = subgraph
    
#     _max = 1 #torch.max(-loss, dim=-1).values.item()
    n_entities = len(entity_ids)
    n_relations = len(predicate_ids)
    predicted = []
    if n_entities == 0 or n_relations == 0:
        return _, _, e_ids, p_ids
    # activations
    x = np.zeros(n_entities)
    for i, e in enumerate(matched_entity_ids):
        if e in entity_ids:
            idx = entity_ids.index(e)
            x[idx] = matched_entity_scores[i]

    p = np.zeros(n_relations)
    for i, e in enumerate(matched_relation_ids):
        if e in predicate_ids:
            idx = predicate_ids.index(e)
            p[idx] = matched_relation_scores[i]

    # load adjacencies
    A = generate_adj_sp(adjacencies, n_entities, include_inverse=True)
    y = np.zeros(n_entities)
    for i, _A in enumerate(A):
        _A = p[i] * _A
        # MP
        _y = x @ _A
        _y[_y>1] = 1
#         _y = p[i] * _y
#             y += _y
#             print(_y)
        y = np.vstack([y, _y])
    y = np.amax(y, 0)
#         print(y)
    answer_scores = []
    for e in (-y).argsort():  # sorts ascending
        # exclude activated entities
        if x[e] == 0:
            answer = kg.global_id_to_string(entity_ids[e], TripleComponentRole.OBJECT)
#             print(answer)
            answer = answer.split('/')[-1]
            if answer in entities:
                predicted.append(entities[answer])
                answer_scores.append(y[e])
    return predicted, answer_scores, e_ids, p_ids
    # DONE

In [6]:
import numpy as np

from utils import preprocess

wav_path = "../data/gtts/sqs_valid_wikidata2018_09_11/"

e_EM, r_EM, a_EM = 0, 0, 0
e_rs, r_rs, a_rs = 0, 0, 0
e_rrs, r_rrs, a_rrs = [], [], []

if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
        
w2v_recognizer.model.to('cuda')
w2v_recognizer.model.eval()

with open('../data/sqs_valid_wikidata2018_09_11.txt') as fin:
    lines = fin.readlines()
    print(len(lines), 'samples')

answer_h1 = 0
answer_h3 = 0
answer_h5 = 0
answer_h10 = 0
entity_recall = 0
property_recall = 0

n = 0

for i, l in enumerate(lines[:]):
#     i = 808
    print(i)
    l = lines[i]
    s, p, o, q = l.strip('\n').split('\t')
    p = p[1:]

    if s in entities and p in properties:
        n += 1
        # load ground truth
        s_label = preprocess(entities[s])
        p_label = preprocess(properties[p])
        a_label = entities[o]

        print(q)
        print(s_label, p_label)

        # predict distribution
        file = wav_path + str(i+1) + '.wav'
        answers, answer_scores, e_ids, p_ids = retrieve_answer(file)
        print(answers[:5], answer_scores[:5])
        print('Entity', s in e_ids)
        print('Property', p in p_ids)

        if answers:
            answer_h1 += int(a_label == answers[0])
            answer_h3 += int(a_label in answers[:3])
            answer_h5 += int(a_label in answers[:5])
            answer_h10 += int(a_label in answers[:10])
            print(answers[0], a_label)
            print('Hit@1', a_label == answers[0])
            print('Hit@5', a_label in answers[:10], '\n')
        entity_recall += int(s in e_ids)
        property_recall += int(p in p_ids)
        
#         break

        
print(n, 'samples')
print('Hits@1:', answer_h1/n)
print('Hits@3:', answer_h3/n)
print('Hits@5:', answer_h5/n)
print('Hits@10:', answer_h10/n)
print('Entity recall:', entity_recall/n)
print('Property recall:', property_recall/n)

Let's use 2 GPUs!
1026 samples
0
where was sasha vujačić born
sasha vujacic place of birth
where was sasha vijasic born
SASHA VUJACIC 1.0
SASHA 0.8902946414888943
THE UNBORN 0.5831051740780094
BASHA 0.5157746571566464
MICHAEL ASHER 0.5130095914112232
BORN AS 1.0
ASASIN 0.6474388822021648
MUSIC PUBLISHER 0.615647035190851
ISIC REV4 0.6014285247253491
BEFORE WAS 0.5715269686683307
['Maribor', 'Kingston', 'London', 'Los Angeles', 'Iraj Janatie Ataie'] [0.46400016246890907, 0.4130968582960461, 0.35624807139938564, 0.2380365337629162, 0.22974674372930862]
Entity True
Property True
Maribor Maribor
Hit@1 True
Hit@5 True 

1
What was Seymour Parker Gilbert's profession?
seymour parker gilbert ocupation
what was semo parkegilbet's profession
SEYMOUR PARKER GILBERT 1.0
THE PROFESIONALS 0.8470345244870595
CRIMES OF PASION 0.4292097551630141
CONFESION 0.4285029921829914
SID PARKER 0.4124555594716818
PROFESION 1.0
PROFESIONAL PARTNER 0.9333134576993175
FIELD OF PROFESION 0.920593573211682
KNOWN PRO

IN AL LANGUAGES 1.0
DAVI BANDA 0.37597612498773875
TONGA 0.29994605552024456
AVINGER 0.2738009807647017
HAVING 0.23208756939396893
LANGUAGES SPOKEN 1.0
LANGUAGE SPOKEN 0.8637474972445253
SPOKE LANGUAGE 0.7426021860104915
LANGUAGE OF SPOKEN TEXT 0.7197895810371043
WRITING LANGUAGES 0.4580220334351487
['English', 'Icelandic', 'Swedish', 'Finnish', 'Ronit Roy'] [0.3305884008921076, 0.19264880181181956, 0.16419843037373727, 0.16419843037373727, 0.11996763540677398]
Entity False
Property True
English Hindi
Hit@1 False
Hit@5 False 

15
What is peter schrøder's profession?
peter schroder ocupation
what is peter shroder's profession
PETER SCHRODER 1.0
PROFESIONAL WRESTLER 0.7912900085470314
THE PROFESIONALS 0.6284258950261101
PROFESOR POTER 0.5081162952025511
PETER YATES 0.3506244083731605
PROFESION 1.0
FIELD OF PROFESION 0.9114260413925352
PROFESIONAL PARTNER 0.8294898178384473
PROFESION'S FIELD 0.8215554235808541
CODE LETERS 0.5421384183407344
['actor', 'screenwriter', 'Bob Orton Jr.', 'Jerr

tommy people's plays what instrument
PERCUSION INSTRUMENT 1.0
TOMY PEOPLES 0.9418827320111189
KEYBOARD INSTRUMENT 0.881391491936734
INSTRUMENTAL ROCK 0.8653661920833389
SINHALA PEOPLE 0.5439729179305285
PLAYS INSTRUMENT 1.0
INSTRUMENT PLAYED 0.8307526999142173
INSTRUMENTATION 0.6788804480061172
INSTRUMENT 0.5724310433955796
MUSICAL INSTRUMENT 0.4293232825466847
['Ruslana', 'Bernard Fanning', 'Ringo Starr', 'John Cale', 'Adriano Celentano'] [1.0, 1.0, 1.0, 1.0, 1.0]
Entity True
Property True
Ruslana fiddle
Hit@1 False
Hit@5 False 

28
what is the country of origin of the tv program gimme a break!
gime a break country of origin
what is the country of origin of the te v programme gimea brake
THE HILO COUNTRY 1.0
PROGRAMER 0.9950152683971045
COUNTRY 0.8156692890001109
COUNTRY TEASERS 0.8069245898077564
COUNTRY HEART 0.8066085342807705
COUNTRY OF ORIGIN 1.0
ORIGIN COUNTRY 0.9014206239912307
PROGRAMER 0.48606952652330065
COUNTRY 0.39845819224720846
PROGRAMED BY 0.3645521448924755
['United St

what is mat duke's profession
PROFESIONAL WRESTLING 1.0
PROFESIONAL WRESTLER 1.0
THE PROFESIONALS 0.884022294394351
MAT DUKE 0.5867639678093418
CONFESION 0.4283658515453485
PROFESION 1.0
FIELD OF THIS PROFESION 0.9806116971908057
FIELD OF PROFESION 0.9190512952934445
PROFESION'S FIELD 0.862178019297266
PROFESIONAL PARTNER 0.7333333333333334
['Bob Orton Jr.', 'Jerry Tuite', 'Hiromichi Fuyuki', 'Orlando Jordan', 'Woody Strode'] [1.0, 1.0, 1.0, 1.0, 1.0]
Entity True
Property True
Bob Orton Jr. singer-songwriter
Hit@1 False
Hit@5 False 

41
What genre is the mr. t experience
the mr t experience genre
what jonre is the mysterty experience
THE MR T EXPERIENCE 1.0
THE BARTON MYSTERY 0.5596086799434523
THE SISTERS 0.3124677824439233
KRISTER HENRIKSON 0.2780251238692475
THE OFENCE 0.2716879104323737
HAS SISTER 1.0
TYPE SYSTEM 0.8219293816615527
SISTER 0.7155596354060392
ISTERS 0.7155596354060392
PERCENT 0.7131789550189304
['punk rock', 'drama', 'novel', 'Bydgoszcz', 'crime film'] [0.43522022208

['English', 'Spanish', 'male', 'female', 'German'] [1.0, 0.393780740468805, 0.17942833939732, 0.1776989466434812, 0.10723099188797403]
Entity True
Property True
English English
Hit@1 True
Hit@5 True 

53
who directed the film three blind mouseketeers
thre blind mouseketers director
who directed the film three blind mousketeers
THRE BLIND MOUSEKETERS 1.0
THE THRE MUSKETERS 0.817791293412146
THE FOUR MUSKETERS 0.5109673793780032
FILM DIRECTOR 0.3552400788361299
THEATER DIRECTOR 0.25935710576265253
LETER DIRECTION 1.0
FILM DIRECTOR 0.9397455719026708
DIRECTED BY 0.7390518162118022
INDIRECT PERPETRATOR 0.6879765529211427
INDIRECTLY CAUSED 0.6726383989388739
['David Hand', 'Paul W. S. Anderson', 'Germany', 'Munich', 'Madrid'] [0.9397455719026708, 0.768515746724622, 0.29658189109337274, 0.29658189109337274, 0.1853084923790927]
Entity True
Property True
David Hand David Hand
Hit@1 True
Hit@5 True 

54
what is the genre of the magic
magic genre
what is the jonere of the magic
THE MAD HATER 1.0

...

what country is langston heres from
LANGSTON HUGHES 1.0
COUNTRY 0.8322871230807829
MOUNT EVEREST 0.7542698826406284
COUNTRY ROCK 0.7376650893129941
CHERY COUNTY 0.7265623662221332
COUNTRY 1.0
AREST COUNT 0.9424014397736219
STORE COUNT 0.9171780569474812
HAS COUNTRIES 0.9073996662999434
COUNTRY CODE 0.8706881025770461
['Pennsylvania', 'United States of America', 'Texas', 'Nepal', 'Nebraska'] [1.0, 1.0, 1.0, 0.7542698826406284, 0.6592824486560609]
Entity True
Property False
Pennsylvania United States of America
Hit@1 False
Hit@5 True 

967
what sport does the team cis play
cis sport
what sport does the team siss play
EA SPORTS 1.0
ESTHERO 0.6693117921859785
WEATHER REPORT 0.650723432133528
PORTER HAL 0.6439189648014388
HEATHROW AIRPORT 0.63264778124093
TEAMS PLAYED FOR 1.0
SPORT TEAM 0.9227109546333507
SPORT PLAYED 0.8894559361696794
TEAM PLAYED FOR 0.7976627063367986
DISPLAYED AT 0.6331344032178726
['baseball', 'association football', 'Los Angeles', 'Jonathan Nichols, Jr.', 'William Pai

what position did mark alrick play
MARK ELRICK 1.0
POSESION 0.5512882769194463
DICK CLARK 0.5306156568105193
MARIO TOSI 0.5301464795047025
MARK DICKSON 0.48024115637634424
PLAYER POSITION 1.0
POLITICAL POSITION 0.831648369129278
POSITION 0.6557191075064449
ANGULAR POSITION 0.6157611179553277
POLE POSITION 0.559211838719801
['forward', 'association football', 'ice hockey', 'cricket', 'keyboard instrument'] [1.0, 0.27016535437677414, 0.054162594867560185, 0.03397913267038968, 0.03143322638272156]
Entity True
Property True
forward forward
Hit@1 True
Hit@5 True 

981
which artist recorded over the edge?
over the edge performer
which artist recorded over the edge
OVER THE EDGE 1.0
VERVE RECORDS 0.9351468084885348
OVER THE REICH 0.8886974277720072
ON THE RECORD 0.7629674882223886
ARISTA RECORDS 0.6521277901206345
COVER ARTIST 1.0
ARTISTIC THEME 0.8380610467788611
RECORDED WITH 0.7464762070713162
ARTISTIC DIRECTOR 0.6785798073016479
RECORDED BY 0.6658677433367376
['jazz', 'World War II', 'Wip

['male', 'female'] [1.0, 0.3363305364996498]
Entity True
Property True
male male
Hit@1 True
Hit@5 True 

994
where in south america is manuel dorrego from?
manuel dorego country of citizenship
wherein south america is manouelde rago from
OLDER THAN AMERICA 1.0
THE AMERICAN GAME 0.7898158959189115
NY MAN IN AMERICA 0.632789360521784
TRANSAMERICA 0.5666862014212508
MERICA FERERA 0.5447023224855854
AMERICAN ART ID 1.0
SOUTHERLY 0.8648676302504862
ISOMERIC SMILES 0.8483907474783747
RAGA THEME 0.6488636275508929
INHERITS FROM 0.6295767285442602
['male', 'female', 'basketball', 'Second Boer War', 'New York City'] [0.4169006860514457, 0.22157468344352454, 0.19907202341806166, 0.03223539065408593, 0.025945539738340767]
Entity False
Property False
male Argentina
Hit@1 False
Hit@5 False 

995
where was samy vellu born
samy velu place of birth
where with samy vally born
BORN IN THE USA 1.0
ARCHER PREWIT 0.9853945384346139
SAMY LERNER 0.7460205722742286
THE UNBORN 0.7251769655990965
SALY WHELER 0.

['Olivia Wilde', 'Richard Blumenthal', 'Anna Kathryn Holbrook', 'Cillian Murphy', 'Morrissey'] [0.4531472321642738, 0.4531472321642738, 0.4531472321642738, 0.4531472321642738, 0.4531472321642738]
Entity True
Property True
Olivia Wilde English
Hit@1 False
Hit@5 False 

1008
who was the cinematography for moon over parador
mon over parador director of photography
here was the sinemotography for mun over paradore
CINEMATOGRAPHER 1.0
MON OVER PARADOR 0.8527628747201281
WAR PHOTOGRAPHER 0.6821371594663905
PHOTOGRAPHER 0.5868083255852711
THORNOGRAPHY 0.5495124088761665
CINEMATOGRAPHER 1.0
CINEMATOGRAPHY 0.9968712851860868
PHOTOGRAPHER 0.5868083255852711
PHOTOGRAPHED WITH 0.43632297230788836
CINEMATOGRAFO ID 0.42867430949501395
['Donald McAlpine', 'José Luis Alcaine Escaño', 'Public Speaking', "Monica's Story", 'The Autobiography of Alice B. Toklas'] [0.8527628747201281, 0.10664879834769136, 0.042932889328417156, 0.027278418073702675, 0.027278418073702675]
Entity True
Property True
Donald McA

['Washington, D.C.', 'Montgomery', 'United States of America', 'Giulio Caccini', 'Jacopo Peri'] [0.7243198371814689, 0.33638680295813195, 0.32656020847057127, 0.2908204960526869, 0.2908204960526869]
Entity True
Property True
Washington, D.C. Washington, D.C.
Hit@1 True
Hit@5 True 

1021
what type of computer videogame genre is the game  red dead redemption: undead nightmare
undead nightmare genre
what type of computer vidio game jonra is the game red dead redemption undead nightmare
UNDEAD NIGHTMARE 1.0
HEADHUNTER REDEMPTION 0.7714712978327373
NIGHTMARE 0.4982105037943172
24 REDEMPTION 0.4709812249949335
THE UNDEAD 0.3224908884916377
TYPE OF DEATH 1.0
AREA OF OPERATIONS 0.7546577263683925
DEATH TYPE 0.7541181860659336
THEATER COMPANY 0.5158039974168748
OPTION 0.44384669457449183
['Imagine Entertainment', 'American International Pictures', 'Paramount Pictures', 'Reliance Entertainment', 'composer'] [0.2429339985606832, 0.16634208941450634, 0.11946416841653006, 0.04233664934577022, 0.023